In [1]:
### Libraries
import json
import time
import xml.etree.ElementTree as ET
import xlwt
from xlwt import Workbook
import csv
import numpy as np
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import requests


##### Data preprocessing chatgpt-3.5

In [2]:
# --- Results chatGPT --- 
article_results_chatGPT = pd.read_csv('./results_chatGPT-3_5/disease/results_chatGPT2_all_newline.csv')
article_results_chatGPT.result = article_results_chatGPT.result.str.replace('\r','')
#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in article_results_chatGPT.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    all_annotations = row[1].replace('\nD0','|D0')
    for chatGPTanno in all_annotations.split('\n'):
        if len(chatGPTanno) > 0:
                chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
    dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
    dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0


# We get the biocreative dataset to compare with our findings
biocreative_disease_corpus = ET.parse('Biocreative_dataset/disease/TestSet.DNorm.BioC.xml')

# find the root
root = biocreative_disease_corpus.getroot()
all_correct_entities = 0
data_frame_articles = pd.DataFrame()
# find all results and convert them into a better format
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_annotations = title_passage.findall('annotation')
    annotations_set = set()
    annotations_set_2 = set()
    for annotation in title_annotations:
        annotations_set.add((annotation.find("text").text,annotation.find('infon[@key="MESH"]').text.replace('MESH:','')))
        annotations_set_2.add(annotation.find("text").text)
    #title_text = title_passage.find('text').text
    abstract_passage = doc.findall('passage')[1]
    abstract_annotations = abstract_passage.findall('annotation')
    for anno in abstract_annotations:
        annotations_set.add((anno.find("text").text,anno.find('infon[@key="MESH"]').text.replace('MESH:','')))
        annotations_set_2.add(anno.find("text").text)
    #abstract_text = abstract_passage.find('text').text
    # we create a list of data
    #array_of_data = [pubmed_id,title_text,abstract_text]
    # append to dataframe
    #data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
    dictionary_of_results[str(pubmed_id)] = annotations_set
    dictionary_of_results_without_MESH[str(pubmed_id)] = annotations_set_2
    all_correct_entities += len(annotations_set_2)
#print(dictionary_of_results)
print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))


path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-3.5/disease/chatGPT_disease_statistics_without_MESH.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-3.5/disease/chatGPT_disease_statistics_with_MESH.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-3.5/disease/confusion_matrix_disease_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/disease/confusion_matrix_disease_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-3.5/disease/confusion_matrix_disease_entities_test_contains_and_chatgpt_contains.csv"



500
500
500
500


### Data preprocessing ChatGPT-4

In [3]:
# --- Results chatGPT --- 
article_results_chatGPT = pd.read_csv('./results_chatGPT-4/disease/TestSet.DNorm.chatGPT4Results.csv')
article_results_chatGPT.result = article_results_chatGPT.result.str.replace('\r','')
#print(article_results_chatGPT)

# we prepare the chatGPT data so that we can easily parse it
dictionary_of_chatGPT_results = {}
dictionary_of_chatGPT_results_without_MESH = {}
for index,row in article_results_chatGPT.iterrows():
    chatGPT_annotations = set()
    chatGPT_annotations_2 = set()
    #all_annotations = row[1].replace('\nD0','|D0')
    if str(row[1]) != "nan":  
        for chatGPTanno in str(row[1]).split('\n'):
            if len(chatGPTanno) > 0:
                    chatGPT_annotations.add((chatGPTanno.strip().split('|')[0],chatGPTanno.strip().split('|')[1]))
                    chatGPT_annotations_2.add(chatGPTanno.strip().split('|')[0])
        dictionary_of_chatGPT_results[str(row[0])] = chatGPT_annotations
        dictionary_of_chatGPT_results_without_MESH[str(row[0])] = chatGPT_annotations_2
    else:
        dictionary_of_chatGPT_results[str(row[0])] = []
        dictionary_of_chatGPT_results_without_MESH[str(row[0])] = []
# test set
correct_entity = 0
incorrect_entity = 0
correct_entity_and_MESH = 0
incorrect_entity_and_MESH = 0


# We get the biocreative dataset to compare with our findings
biocreative_disease_corpus = ET.parse('Biocreative_dataset/disease/TestSet.DNorm.BioC.xml')

# find the root
root = biocreative_disease_corpus.getroot()
all_correct_entities = 0
data_frame_articles = pd.DataFrame()
# find all results and convert them into a better format
dictionary_of_results = {}
dictionary_of_results_without_MESH = {}
for doc in root.findall('document'):
    # Get the pubmed id, title and abstract text
    pubmed_id = doc.find('id').text
    title_passage = doc.findall('passage')[0]
    title_annotations = title_passage.findall('annotation')
    annotations_set = set()
    annotations_set_2 = set()
    for annotation in title_annotations:
        annotations_set.add((annotation.find("text").text,annotation.find('infon[@key="MESH"]').text.replace('MESH:','')))
        annotations_set_2.add(annotation.find("text").text)
    #title_text = title_passage.find('text').text
    abstract_passage = doc.findall('passage')[1]
    abstract_annotations = abstract_passage.findall('annotation')
    for anno in abstract_annotations:
        annotations_set.add((anno.find("text").text,anno.find('infon[@key="MESH"]').text.replace('MESH:','')))
        annotations_set_2.add(anno.find("text").text)
    #abstract_text = abstract_passage.find('text').text
    # we create a list of data
    #array_of_data = [pubmed_id,title_text,abstract_text]
    # append to dataframe
    #data_frame_articles = data_frame_articles.append(pd.Series(array_of_data,index=["pubmed_id","title","abstract"]),ignore_index=True)
    dictionary_of_results[str(pubmed_id)] = annotations_set
    dictionary_of_results_without_MESH[str(pubmed_id)] = annotations_set_2
    all_correct_entities += len(annotations_set_2)
#print(dictionary_of_results)
print(len(dictionary_of_chatGPT_results))
print(len(dictionary_of_results))
print(len(dictionary_of_chatGPT_results_without_MESH))
print(len(dictionary_of_results_without_MESH))
path_to_results_file_without_mesh = "basic_occurences_results/chatgpt-4/disease/chatGPT_disease_statistics_without_MESH.txt"
path_to_results_file_with_mesh = "basic_occurences_results/chatgpt-4/disease/chatGPT_disease_statistics_with_MESH.txt"
path_to_confusion_matrix_pure = "confusion_matrix_results_CHATGPT/chatgpt-4/disease/confusion_matrix_disease_pure_entities.csv"
path_to_confusion_matrix_partial_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/disease/confusion_matrix_disease_entities_test_contains.csv"
path_to_confusion_matrix_partial_full_lapover = "confusion_matrix_results_CHATGPT/chatgpt-4/disease/confusion_matrix_disease_entities_test_contains_and_chatgpt_contains.csv"

500
500
500
500


# Analysis of the data (Only entities)
_______

### PURE CORRECT ENTITIES

In [18]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper():
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 1217
All correct entities: 2551
Percentage(%): 47.70
Number of excess entities (incorrect): 963


### If the test entity is contained in the CHATGPT entity

In [19]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper():
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 1638
All correct entities: 2551
Percentage(%): 64.21
Number of excess entities (incorrect): 542


### If test entities can be found in CHATGPTS response and vice versa

In [20]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results_without_MESH[key]
    chatGPT_results = dictionary_of_chatGPT_results_without_MESH[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual.strip().upper() == test.upper() or test.upper() in actual.strip().upper() or actual.strip().upper() in test.upper():
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_without_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 1755
All correct entities: 2551
Percentage(%): 68.79
Number of excess entities (incorrect): 425


# Analysis of data (with MESH)
____

## Pure correct entities

In [21]:
# analysis variables
pure_correct_entity = 0
incorrect_entity = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# Pure correct
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if actual[0].strip().upper() == test[0].upper() and actual[1] == test[1]:
                pure_correct_entity += 1
                counter += 1
                break
                
    incorrect_entity += length_of_test - counter

# Pure correct Entities
print("Pure correct entities: "+str(pure_correct_entity))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((pure_correct_entity/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** PURE ENTITIES ***\n\nPure correct entities: '+str(pure_correct_entity)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((pure_correct_entity/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity)+'\n\n')
file1.close()

Pure correct entities: 552
All correct entities: 2551
Percentage(%): 21.63
Number of excess entities (incorrect): 1629


### If the test entity is contained in the CHATGPT entity

In [22]:
# analysis variables
correct_entity_with_test_contain = 0
incorrect_entity_with_test_contain = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper()) and (actual[1] == test[1]):
                correct_entity_with_test_contain += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain: "+str(correct_entity_with_test_contain))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response ***\n\ncorrect entities: '+str(correct_entity_with_test_contain)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain)+'\n\n')
file1.close()

correct entities with test contain: 626
All correct entities: 2551
Percentage(%): 24.53
Number of excess entities (incorrect): 1555


### If test entities can be found in CHATGPTS response and vice versa

In [23]:
# analysis variables
correct_entity_with_test_contain_and_vice_versa = 0
incorrect_entity_with_test_contain_and_vice_versa = 0

#print(dictionary_of_results)
#print(dictionary_of_chatGPT_results_without_MESH)
# First we check how many entities were correct

# correct Entities with test contained in CHATGPT response for loop
for key in dictionary_of_results_without_MESH:
    test_results = dictionary_of_results[key]
    chatGPT_results = dictionary_of_chatGPT_results[key]
    length_of_test = len(chatGPT_results)
    counter = 0
    for test in test_results:
        for actual in chatGPT_results:
            if (actual[0].strip().upper() == test[0].upper() or test[0].upper() in actual[0].strip().upper() or actual[0].strip().upper() in test[0].upper()) and (actual[1] == test[1]):
                correct_entity_with_test_contain_and_vice_versa += 1
                counter += 1
                break
                
    incorrect_entity_with_test_contain_and_vice_versa += length_of_test - counter

# correct Entities with test contained in CHATGPT response analysis
print("correct entities with test contain and vice versa: "+str(correct_entity_with_test_contain_and_vice_versa))
print("All correct entities: "+str(all_correct_entities))
print("Percentage(%): "+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5])
print("Number of excess entities (incorrect): "+str(incorrect_entity_with_test_contain_and_vice_versa))

file1 = open(path_to_results_file_with_mesh, "a")
file1.write('*** ENTITIES from test are contained in chatGPT response and vice versa ***\n\ncorrect entities: '+str(correct_entity_with_test_contain_and_vice_versa)+'\nAll correct entities: '+str(all_correct_entities)+'\nPercentage(%): '+str((correct_entity_with_test_contain_and_vice_versa/all_correct_entities)*100)[0:5]+'\nNumber of excess entities (incorrect): '+str(incorrect_entity_with_test_contain_and_vice_versa)+'\n\n')
file1.close()

correct entities with test contain and vice versa: 682
All correct entities: 2551
Percentage(%): 26.73
Number of excess entities (incorrect): 1499


# Analysis confusion matrix
_____

## Preprocessing

In [24]:
### Create correct dictionaries
pairs_test_set = set()
pairs_chatGPT_set = []
unique_chatGPT_ids = set()
unique_test_set = set()

# get all meshes from CHATGPT and their entities
for key in dictionary_of_chatGPT_results:
    for mesh_id in dictionary_of_chatGPT_results[key]:
        pairs_chatGPT_set.append((mesh_id[0],mesh_id[1]))
        unique_chatGPT_ids.add(mesh_id[1])

# get all unique meshes from test set
for key in dictionary_of_results:
    for mesh_id in dictionary_of_results[key]:
        pairs_test_set.add((mesh_id[0],mesh_id[1]))
        unique_test_set.add(mesh_id[1])

list_of_test_meshes = list(pairs_test_set)
unique_test_set = list(unique_test_set)
list_of_chatGPT_MESHES = list(unique_chatGPT_ids)

### Function to create dataframe

In [25]:
def createRowForDataframe(key,array,all_chatGPT_unique_ids):
    array_of_values = {}
    array_of_values["test_MESH_ID"] = key 
    for element in all_chatGPT_unique_ids:
        counter = 0
        for ground_base_id in array:
            if element == ground_base_id:
                counter += 1
        array_of_values[element] = counter
    return array_of_values



## Pure correct entities

In [26]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0] == entity_set[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17168\1577645711.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\1577645711.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\1577645711.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matri

#### Clean confusion matrix and save it into a csv

In [27]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_pure,index=False)

## If the test entity is contained in the CHATGPT entity

In [28]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17168\2444638993.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\2444638993.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\2444638993.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matri

#### Clean confusion matrix and save it into a csv

In [29]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_lapover,index=False)

### If test entities can be found in CHATGPTS response and vice versa

In [30]:
dictionary_of_matrix_occurences = {}
# Now we need to loop through all CHATGPT responses with each test mesh
for test_entity in list_of_test_meshes:
    array_of_mesh_ids = []
    for entity_set in pairs_chatGPT_set:
        if test_entity[0] == entity_set[0] or test_entity[0] in entity_set[0] or entity_set[0] in test_entity[0]:
            array_of_mesh_ids.append(entity_set[1])
    dictionary_of_matrix_occurences[test_entity[1]] = array_of_mesh_ids
    


confusion_matrix = pd.DataFrame(columns=['test_MESH_ID']+list_of_chatGPT_MESHES)
# now we create rows for dataFrames
for keys in dictionary_of_matrix_occurences:
    confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)


C:\Users\matev\AppData\Local\Temp\ipykernel_17168\3431490947.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\3431490947.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matrix_occurences[keys],list_of_chatGPT_MESHES),ignore_index=True)
C:\Users\matev\AppData\Local\Temp\ipykernel_17168\3431490947.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  confusion_matrix = confusion_matrix.append(createRowForDataframe(keys,dictionary_of_matri

#### Clean confusion matrix and save it into a csv

In [31]:
# Check if each column contains only zeros
zero_columns = confusion_matrix.columns[confusion_matrix.eq(0).all()]
# Drop the zero columns from the dataframe
confusion_matrix = confusion_matrix.drop(zero_columns, axis=1)

# Get the subset of columns excluding the 'ID' column
columns_to_check = confusion_matrix.columns[~(confusion_matrix.columns == 'test_MESH_ID')]

# Check if each row (excluding the 'ID' column) contains only zeros
zero_rows = confusion_matrix.loc[(confusion_matrix[columns_to_check] == 0).all(axis=1)]

# Drop the zero rows from the dataframe
confusion_matrix = confusion_matrix.drop(zero_rows.index)

confusion_matrix.to_csv(path_to_confusion_matrix_partial_full_lapover,index=False)